# This notebook compiles a master dataset of the most recently available training data with respect to both deaths-related and county-specific data that we feel is likely most correlated with COVID-19 death figures

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import git

In [2]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [3]:
# import daily dataset
dfd = pd.read_csv(f"{homedir}/data/us/covid/nyt_us_counties_daily.csv", converters={'fips' : str})

In [4]:
# get all counties' fips that require predictions
counties = list(set(list(dfd['fips'])))

In [5]:
# compile initial training dataframe; only including county/date entries after county's date of first death
from tqdm import tqdm

dfX = pd.DataFrame(columns=dfd.columns)
num_counties = len(counties)

for county in tqdm(counties):
    entries = dfd[dfd['fips'] == county]
    start = False
    for index, row in entries.iterrows():
        if row['deaths'] > 0:
            start = True
        if start:
            dfX = dfX.append(row)

100%|██████████| 2937/2937 [07:49<00:00,  6.26it/s]


In [6]:
dfX.reset_index(inplace=True)
dfX.head()

,index,fips,date,county,state,cases,deaths
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0


In [7]:
# import cumulative dataset
dfc = pd.read_csv(f"{homedir}/data/us/covid/nyt_us_counties.csv", converters={'fips' : str})

In [8]:
# compile entry specific data constructable from current and cumulative dataframes

import datetime as dt

# string to datetime object conversion
def date_from_str(ds):
    year = int(ds[:4])
    month = int(ds[5:7])
    day = int(ds[8:])
    return dt.datetime(year, month, day)

# datetime object to string conversion
def str_from_date(date):
    return date.strftime("%Y-%m-%d")

# returns date that is N weeks prior to date
def weeks_prior_date(date, N):
    return (date - dt.timedelta(weeks=N))

# days since date of first death
ddfd = []
# day of week
dw = []
# day of month
dm = []
# deaths one, two, three, four, and five day(s) prior (non-cumulative)
d1 = []
d2 = []
d3 = []
d4 = []
d5 = []
# cases one, two, three, four, five, and six weeks prior (cumulative)
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []

dates = np.array(dfX['date'])
Xcounties = np.array(dfX['fips'])
deaths = np.array(dfX['deaths'])
curr_county = ""
prev_county = ""
curr_ddfd = 0
curr_date = ""
curr_d1 = 0
curr_d2 = 0
curr_d3 = 0
curr_d4 = 0
curr_d5 = 0
for i in tqdm(range(len(dfX))):
    prev_county = curr_county
    curr_county = Xcounties[i]
    if prev_county != curr_county:
        curr_ddfd = 0
        curr_d1 = 0
        curr_d2 = 0
        curr_d3 = 0
        curr_d4 = 0
        curr_d5 = 0
    
    # deaths related features
    curr_deaths = deaths[i]
    d1.append(curr_d1)
    d2.append(curr_d2)
    d3.append(curr_d3)
    d4.append(curr_d4)
    d5.append(curr_d5)
    
    # date related features
    curr_date = date_from_str(dates[i])
    dw.append(curr_date.weekday())
    dm.append(curr_date.day)
    ddfd.append(curr_ddfd)
    
    # case related features
    date_1wp = str_from_date(weeks_prior_date(curr_date, 1))
    date_2wp = str_from_date(weeks_prior_date(curr_date, 2))
    date_3wp = str_from_date(weeks_prior_date(curr_date, 3))
    date_4wp = str_from_date(weeks_prior_date(curr_date, 4))
    date_5wp = str_from_date(weeks_prior_date(curr_date, 5))
    date_6wp = str_from_date(weeks_prior_date(curr_date, 6))
    
    df_1wp = dfc.loc[(dfc['date'] == date_1wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_1wp.empty:
        c1.append(float(df_1wp['cases'][0]))
    else:
        c1.append(0.0)
    
    df_2wp = dfc.loc[(dfc['date'] == date_2wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_2wp.empty:
        c2.append(float(df_2wp['cases'][0]))
    else:
        c2.append(0.0)
    
    df_3wp = dfc.loc[(dfc['date'] == date_3wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_3wp.empty:
        c3.append(float(df_3wp['cases'][0]))
    else:
        c3.append(0.0)
        
    df_4wp = dfc.loc[(dfc['date'] == date_4wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_4wp.empty:
        c4.append(float(df_4wp['cases'][0]))
    else:
        c4.append(0.0)
        
    df_5wp = dfc.loc[(dfc['date'] == date_5wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_5wp.empty:
        c5.append(float(df_5wp['cases'][0]))
    else:
        c5.append(0.0)
        
    df_6wp = dfc.loc[(dfc['date'] == date_6wp) & (dfc['fips'] == curr_county)].reset_index()
    if not df_6wp.empty:
        c6.append(float(df_6wp['cases'][0]))
    else:
        c6.append(0.0)
        
    # update features for next entry
    curr_d5 = curr_d4
    curr_d4 = curr_d3
    curr_d3 = curr_d2
    curr_d2 = curr_d1
    curr_d1 = curr_deaths
    curr_ddfd += 1

100%|██████████| 73773/73773 [2:30:40<00:00,  8.16it/s]  


In [9]:
# add entry specific data to training dataframe
dfX['DDFD'] = np.array(ddfd)
dfX['DW'] = np.array(dw)
dfX['DM'] = np.array(dm)
dfX['D1DP'] = np.array(d1)
dfX['D2DP'] = np.array(d2)
dfX['D3DP'] = np.array(d3)
dfX['D4DP'] = np.array(d4)
dfX['D5DP'] = np.array(d5)
dfX['C1WP'] = np.array(c1)
dfX['C2WP'] = np.array(c2)
dfX['C3WP'] = np.array(c3)
dfX['C4WP'] = np.array(c4)
dfX['C5WP'] = np.array(c5)
dfX['C6WP'] = np.array(c6)

In [10]:
dfX[dfX['fips'] == '53061']

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,D2DP,D3DP,D4DP,D5DP,C1WP,C2WP,C3WP,C4WP,C5WP,C6WP
20010,50,53061,2020-03-11,Snohomish,Washington,8.0,1.0,0,2,11,...,0.0,0.0,0.0,0.0,8.0,1.0,1.0,1.0,1.0,1.0
20011,51,53061,2020-03-12,Snohomish,Washington,38.0,2.0,1,3,12,...,0.0,0.0,0.0,0.0,17.0,1.0,1.0,1.0,1.0,1.0
20012,52,53061,2020-03-13,Snohomish,Washington,25.0,0.0,2,4,13,...,1.0,0.0,0.0,0.0,18.0,2.0,1.0,1.0,1.0,1.0
20013,53,53061,2020-03-14,Snohomish,Washington,22.0,0.0,3,5,14,...,2.0,1.0,0.0,0.0,26.0,2.0,1.0,1.0,1.0,1.0
20014,54,53061,2020-03-15,Snohomish,Washington,21.0,0.0,4,6,15,...,0.0,2.0,1.0,0.0,36.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20079,119,53061,2020-05-19,Snohomish,Washington,28.0,6.0,69,1,19,...,0.0,0.0,4.0,2.0,2998.0,2807.0,2370.0,2162.0,1949.0,1658.0
20080,120,53061,2020-05-20,Snohomish,Washington,9.0,0.0,70,2,20,...,0.0,0.0,0.0,4.0,3009.0,2830.0,2416.0,2208.0,1987.0,1694.0
20081,121,53061,2020-05-21,Snohomish,Washington,36.0,1.0,71,3,21,...,6.0,0.0,0.0,0.0,3048.0,2889.0,2446.0,2239.0,2031.0,1701.0
20082,122,53061,2020-05-22,Snohomish,Washington,14.0,2.0,72,4,22,...,0.0,6.0,0.0,0.0,3065.0,2917.0,2466.0,2267.0,2055.0,1801.0


In [11]:
# import aggregate Berkeley and JHU datasets
dfb = pd.read_csv(f"{homedir}/data/us/aggregate_berkeley.csv", converters={'countyFIPS' : int})
dfj = pd.read_csv(f"{homedir}/data/us/aggregate_jhu.csv", converters={'FIPS' : int})

In [12]:
# compile county specific data from Berkeley and JHU dataframes

# filter dataframes for desired features
b_cols_include = ['countyFIPS', 'PopulationEstimate2018', 'FracMale2017',
                  'PopulationEstimate65+2017', 'PopulationDensityperSqMile2010', 'DiabetesPercentage', 
                  'HeartDiseaseMortality', 'StrokeMortality', 'Smokers_Percentage', 
                  '#FTEHospitalTotal2017', '#Hospitals', '#ICU_beds', 'dem_to_rep_ratio', 
                  'PopMale10-142010', 'PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010', 
                  'PopMale20-242010', 'PopFmle20-242010', 'PopMale25-292010', 'PopFmle25-292010', 
                  'PopMale45-542010', 'PopFmle45-542010', 'PopMale55-592010', 'PopFmle55-592010', 
                  'PopMale60-642010', 'PopFmle60-642010', 'PopMale65-742010', 'PopFmle65-742010', 
                  'PopMale75-842010', 'PopFmle75-842010', 'PopMale>842010', 'PopFmle>842010', 
                  '3-YrMortalityAge45-54Years2015-17', '3-YrMortalityAge55-64Years2015-17', 
                  '3-YrMortalityAge65-74Years2015-17', '3-YrMortalityAge75-84Years2015-17', 
                  '3-YrMortalityAge85+Years2015-17']
dfb = dfb[b_cols_include]

j_cols_include = ['FIPS', 'Economic_typology_2015', 'POVALL_2018', 'PCTPOVALL_2018', 
                  'Civilian_labor_force_2018', 'Jan Precipitation / inch', 
                  'Feb Precipitation / inch', 'Mar Precipitation / inch', 
                  'Apr Precipitation / inch', 'May Precipitation / inch', 
                  'Jun Precipitation / inch', 'Jul Precipitation / inch', 
                  'Jan Temp AVG / F', 'Feb Temp AVG / F', 'Mar Temp AVG / F', 
                  'Apr Temp AVG / F', 'May Temp AVG / F', 'Jun Temp AVG / F', 
                  'Jul Temp AVG / F', 'Housing units', 'Total households!!Family households (families)', 
                  'Total households!!Households with one or more people 65 years and over', 
                  'Total households!!Average household size', 'RELATIONSHIP!!Population in households', 
                  'Cardiovascular Disease (AAMC)', 
                  'Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC)', 
                  'Active Physicians per 100000 Population 2018 (AAMC)']
dfj = dfj[j_cols_include]

In [13]:
dfb.head()

,countyFIPS,PopulationEstimate2018,FracMale2017,PopulationEstimate65+2017,PopulationDensityperSqMile2010,DiabetesPercentage,HeartDiseaseMortality,StrokeMortality,Smokers_Percentage,#FTEHospitalTotal2017,...,PopFmle65-742010,PopMale75-842010,PopFmle75-842010,PopMale>842010,PopFmle>842010,3-YrMortalityAge45-54Years2015-17,3-YrMortalityAge55-64Years2015-17,3-YrMortalityAge65-74Years2015-17,3-YrMortalityAge75-84Years2015-17,3-YrMortalityAge85+Years2015-17
0,1001,55601.0,0.486578,8392.0,91.8,9.9,204.5,56.1,18.081557,324.0,...,2192.0,841.0,1141.0,159.0,392.0,44.0,73.0,112.0,143.0,109.0
1,1003,218022.0,0.485472,42413.0,114.7,8.5,183.2,41.9,17.489033,2101.0,...,9245.0,4379.0,5153.0,1176.0,2057.0,125.0,262.0,434.0,546.0,597.0
2,1005,24881.0,0.527701,4757.0,31.0,15.7,220.4,49.0,21.999985,181.0,...,1171.0,498.0,730.0,129.0,314.0,21.0,44.0,65.0,72.0,68.0
3,1007,22400.0,0.535469,3632.0,36.8,13.3,225.5,57.2,19.114200,145.0,...,920.0,370.0,534.0,73.0,206.0,24.0,37.0,62.0,56.0,45.0
4,1009,57840.0,0.493114,10351.0,88.9,14.9,224.8,52.8,19.208672,103.0,...,2688.0,1102.0,1450.0,234.0,574.0,49.0,94.0,150.0,182.0,142.0


In [14]:
dfj.head()

,FIPS,Economic_typology_2015,POVALL_2018,PCTPOVALL_2018,Civilian_labor_force_2018,Jan Precipitation / inch,Feb Precipitation / inch,Mar Precipitation / inch,Apr Precipitation / inch,May Precipitation / inch,...,Jun Temp AVG / F,Jul Temp AVG / F,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC)
0,0,NaN,41852315.0,13.1,161640488.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,131704730.0,66732004.0,30084341.0,0.00,271946728.0,NaN,NaN,NaN
1,1000,NaN,801758.0,16.8,2198837.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2171853.0,NaN,NaN,NaN,NaN,292.000000,0.344,217.1
2,1001,0.0,7587.0,13.8,25957.0,6.78,3.75,3.88,5.17,3.65,...,79.5,81.6,22135.0,NaN,NaN,NaN,NaN,3.321588,0.344,217.1
3,1003,5.0,21069.0,9.8,93849.0,3.60,2.84,2.53,6.45,3.75,...,80.8,81.9,104061.0,55354.0,29568.0,2.57,214759.0,13.024571,0.344,217.1
4,1005,3.0,6788.0,30.9,8373.0,5.86,1.71,3.48,4.71,2.17,...,79.2,80.8,11829.0,NaN,NaN,NaN,NaN,1.486384,0.344,217.1


In [15]:
dfj.append(pd.Series(dtype=float), ignore_index=True).tail()

,FIPS,Economic_typology_2015,POVALL_2018,PCTPOVALL_2018,Civilian_labor_force_2018,Jan Precipitation / inch,Feb Precipitation / inch,Mar Precipitation / inch,Apr Precipitation / inch,May Precipitation / inch,...,Jun Temp AVG / F,Jul Temp AVG / F,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC)
3269,72147.0,NaN,NaN,NaN,2734.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5541.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,72149.0,NaN,NaN,NaN,7329.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3271,72151.0,NaN,NaN,NaN,8826.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15798.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3272,72153.0,NaN,NaN,NaN,9770.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,18020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# build Berkeley and JHU training sub-dataframes separately
dfXb = pd.DataFrame(columns=dfb.columns)
dfXj = pd.DataFrame(columns=dfj.columns)
for county in tqdm(dfX['fips']):
    rowb = dfb[dfb['countyFIPS'] == int(county)]
    if not rowb.empty:
        for index, row in rowb.iterrows():
            dfXb = dfXb.append(row)
            break
    else:
        dfXb = dfXb.append(pd.Series(dtype=float), ignore_index=True)
    
    rowj = dfj[dfj['FIPS'] == int(county)]
    if not rowj.empty:
        for index, row in rowj.iterrows():
            dfXj = dfXj.append(row)
            break
    else:
        dfXj = dfXj.append(pd.Series(dtype=float), ignore_index=True)

100%|██████████| 73773/73773 [12:17<00:00, 99.97it/s] 


In [17]:
dfXj

,FIPS,Economic_typology_2015,POVALL_2018,PCTPOVALL_2018,Civilian_labor_force_2018,Jan Precipitation / inch,Feb Precipitation / inch,Mar Precipitation / inch,Apr Precipitation / inch,May Precipitation / inch,...,Jun Temp AVG / F,Jul Temp AVG / F,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC)
0,6023.0,0.0,27002.0,20.3,63027.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
1,6023.0,0.0,27002.0,20.3,63027.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
2,6023.0,0.0,27002.0,20.3,63027.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
3,45063.0,0.0,36432.0,12.5,147320.0,2.07,2.52,0.80,2.09,2.08,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
4,45063.0,0.0,36432.0,12.5,147320.0,2.07,2.52,0.80,2.09,2.08,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,13185.0,0.0,28539.0,25.9,51746.0,2.62,2.48,3.35,2.42,11.18,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
490,13185.0,0.0,28539.0,25.9,51746.0,2.62,2.48,3.35,2.42,11.18,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
490,13185.0,0.0,28539.0,25.9,51746.0,2.62,2.48,3.35,2.42,11.18,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
490,13185.0,0.0,28539.0,25.9,51746.0,2.62,2.48,3.35,2.42,11.18,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7


In [18]:
dfXb

,countyFIPS,PopulationEstimate2018,FracMale2017,PopulationEstimate65+2017,PopulationDensityperSqMile2010,DiabetesPercentage,HeartDiseaseMortality,StrokeMortality,Smokers_Percentage,#FTEHospitalTotal2017,...,PopFmle65-742010,PopMale75-842010,PopFmle75-842010,PopMale>842010,PopFmle>842010,3-YrMortalityAge45-54Years2015-17,3-YrMortalityAge55-64Years2015-17,3-YrMortalityAge65-74Years2015-17,3-YrMortalityAge75-84Years2015-17,3-YrMortalityAge85+Years2015-17
0,6023.0,136373.0,0.498113,23544.0,37.7,7.9,175.0,66.0,14.468508,1935.0,...,5010.0,2358.0,3131.0,854.0,1711.0,87.0,214.0,282.0,271.0,367.0
1,6023.0,136373.0,0.498113,23544.0,37.7,7.9,175.0,66.0,14.468508,1935.0,...,5010.0,2358.0,3131.0,854.0,1711.0,87.0,214.0,282.0,271.0,367.0
2,6023.0,136373.0,0.498113,23544.0,37.7,7.9,175.0,66.0,14.468508,1935.0,...,5010.0,2358.0,3131.0,854.0,1711.0,87.0,214.0,282.0,271.0,367.0
3,45063.0,295032.0,0.487053,44629.0,375.4,10.6,162.5,36.5,16.304205,4707.0,...,9878.0,3931.0,5609.0,1137.0,2587.0,172.0,363.0,501.0,593.0,652.0
4,45063.0,295032.0,0.487053,44629.0,375.4,10.6,162.5,36.5,16.304205,4707.0,...,9878.0,3931.0,5609.0,1137.0,2587.0,172.0,363.0,501.0,593.0,652.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,13185.0,116321.0,0.487233,13808.0,220.2,16.2,200.1,43.2,19.849227,2710.0,...,3397.0,1366.0,2005.0,319.0,825.0,73.0,141.0,178.0,205.0,189.0
452,13185.0,116321.0,0.487233,13808.0,220.2,16.2,200.1,43.2,19.849227,2710.0,...,3397.0,1366.0,2005.0,319.0,825.0,73.0,141.0,178.0,205.0,189.0
452,13185.0,116321.0,0.487233,13808.0,220.2,16.2,200.1,43.2,19.849227,2710.0,...,3397.0,1366.0,2005.0,319.0,825.0,73.0,141.0,178.0,205.0,189.0
452,13185.0,116321.0,0.487233,13808.0,220.2,16.2,200.1,43.2,19.849227,2710.0,...,3397.0,1366.0,2005.0,319.0,825.0,73.0,141.0,178.0,205.0,189.0


In [19]:
# add county specific data to training dataframe

# combine Berkeley and JHU training sub-dataframes and concatenate to training dataframe
dfX = pd.concat([dfX.reset_index(drop=True), 
           (pd.concat([dfXb.reset_index(drop=True), dfXj.reset_index(drop=True)], axis=1))], axis=1)

In [20]:
dfX

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,Jun Temp AVG / F,Jul Temp AVG / F,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC)
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0,0,3,21,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0,1,4,22,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0,2,5,23,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0,0,0,16,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0,1,1,17,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73768,13947,13185,2020-05-19,Lowndes,Georgia,1.0,0.0,53,1,19,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73769,13948,13185,2020-05-20,Lowndes,Georgia,17.0,0.0,54,2,20,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73770,13949,13185,2020-05-21,Lowndes,Georgia,3.0,0.0,55,3,21,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73771,13950,13185,2020-05-22,Lowndes,Georgia,3.0,0.0,56,4,22,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7


In [21]:
dfX.drop(['FIPS', 'countyFIPS'], axis=1)

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,Jun Temp AVG / F,Jul Temp AVG / F,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC)
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0,0,3,21,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0,1,4,22,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0,2,5,23,...,NaN,NaN,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0,0,0,16,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0,1,1,17,...,61.8,66.4,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73768,13947,13185,2020-05-19,Lowndes,Georgia,1.0,0.0,53,1,19,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73769,13948,13185,2020-05-20,Lowndes,Georgia,17.0,0.0,54,2,20,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73770,13949,13185,2020-05-21,Lowndes,Georgia,3.0,0.0,55,3,21,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7
73771,13950,13185,2020-05-22,Lowndes,Georgia,3.0,0.0,56,4,22,...,70.3,76.1,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7


In [22]:
print(len(dfX.columns))

86


In [23]:
# save current data in case of notebook failure
dfX.to_csv(index=False, path_or_buf="incomplete_master_X.csv")

In [24]:
# import geographical county population centers (latitude and longitude)
dfg = pd.read_csv(f"{homedir}/data/us/geolocation/county_centers.csv", converters={'fips' : int})
dfg.head()

,fips,clon00,clat00,clon10,clat10,pclon00,pclat00,pclon10,pclat10
0,1001,-86.577176,32.523283,-86.644490,32.536382,-86.501832,32.500323,-86.494165,32.500389
1,1003,-87.748260,30.592781,-87.746067,30.659218,-87.760540,30.565383,-87.762381,30.548923
2,1005,-85.331312,31.856515,-85.405456,31.870670,-85.306746,31.847869,-85.310038,31.844036
3,1007,-87.123243,33.040054,-87.127148,33.015893,-87.127019,33.025947,-87.127659,33.030921
4,1009,-86.554768,33.978461,-86.567246,33.977448,-86.582617,33.962601,-86.591491,33.955243


In [25]:
# build lists of pclon10 and pclat10 (population-weighted center coordinates) and add to training dataframe
pclon10 = []
pclat10 = []
for county in tqdm(dfX['fips']):
    rowg = dfg[dfg['fips'] == int(county)]
    if not rowg.empty:
        for index, row in rowb.iterrows():
            pclon10.append(float(dfg[dfg['fips'] == int(county)]['pclon10']))
            pclat10.append(float(dfg[dfg['fips'] == int(county)]['pclat10']))
            break
    else:
        pclon10.append(float('NaN'))
        pclat10.append(float('NaN'))
dfX['pclon10'] = np.array(pclon10)
dfX['pclat10'] = np.array(pclat10)

100%|██████████| 73773/73773 [02:06<00:00, 585.07it/s]


In [26]:
# drop unnecessary extra fips columns
dfX= dfX.drop(['FIPS', 'countyFIPS'], axis=1)

In [27]:
dfX

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,Housing units,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC),pclon10,pclat10
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0,0,3,21,...,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0,1,4,22,...,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0,2,5,23,...,61559.0,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0,0,0,16,...,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0,1,1,17,...,113957.0,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73768,13947,13185,2020-05-19,Lowndes,Georgia,1.0,0.0,53,1,19,...,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006
73769,13948,13185,2020-05-20,Lowndes,Georgia,17.0,0.0,54,2,20,...,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006
73770,13949,13185,2020-05-21,Lowndes,Georgia,3.0,0.0,55,3,21,...,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006
73771,13950,13185,2020-05-22,Lowndes,Georgia,3.0,0.0,56,4,22,...,43921.0,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006


In [28]:
# import state reopening dates
dfr = pd.read_csv(f"{homedir}/our_data/state_reopenings.csv", 
                  converters={'State' : str, 'Open_Date' : str})
dfr.head()

,State,Open_Date
0,Alabama,2020-04-30
1,Alaska,2020-03-28
2,Arizona,2020-05-15
3,Arkansas,2020-05-06
4,California,NA


In [29]:
dfr

,State,Open_Date
0,Alabama,2020-04-30
1,Alaska,2020-03-28
2,Arizona,2020-05-15
3,Arkansas,2020-05-06
4,California,NA
5,Colorado,2020-04-26
6,Connecticut,2020-05-20
7,Delaware,2020-05-15
8,Florida,2020-05-04
9,Georgia,2020-04-30


In [30]:
# add state reopening dates to training dataframe in the form of days since or until the reopening date;
# days until will be recognized as a negative number of days
# also I just discovered df.at[] - definitely could have used this throughout all my prior projects and this one
reopenings = []
for i in tqdm(range(len(dfX))):
    state = str(dfX.at[i, 'state'])
    date = date_from_str(str(dfX.at[i, 'date']))
    
    rowr = dfr[dfr['State'] == state]
    if not rowr.empty:
        reo_date = np.array(rowr['Open_Date'])[0]
        if reo_date == 'NA':
            reopenings.append(float('NaN'))
        else:
            reopenings.append(float((date - date_from_str(reo_date)).days))
    else:
        reopenings.append(float('NaN'))
dfX['Days from Reopening'] = reopenings

100%|██████████| 73773/73773 [00:36<00:00, 2040.15it/s]


In [31]:
dfX

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,Total households!!Family households (families),Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC),pclon10,pclat10,Days from Reopening
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0,0,3,21,...,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0,1,4,22,...,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0,2,5,23,...,29581.0,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0,0,0,16,...,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732,-49.0
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0,1,1,17,...,76627.0,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732,-48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73768,13947,13185,2020-05-19,Lowndes,Georgia,1.0,0.0,53,1,19,...,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,19.0
73769,13948,13185,2020-05-20,Lowndes,Georgia,17.0,0.0,54,2,20,...,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,20.0
73770,13949,13185,2020-05-21,Lowndes,Georgia,3.0,0.0,55,3,21,...,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,21.0
73771,13950,13185,2020-05-22,Lowndes,Georgia,3.0,0.0,56,4,22,...,23712.0,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,22.0


In [32]:
# import DL mobility data
dfm = pd.read_csv(f"{homedir}/data/us/mobility/DL-us-mobility-daterow.csv", 
                  converters={'date' : str, 'm50_index' : float, 'fips' : str})
dfm.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,US,1,Alabama,NaN,01,133826,8.331,79.0
1,2020-03-02,US,1,Alabama,NaN,01,143632,10.398,98.0
2,2020-03-03,US,1,Alabama,NaN,01,146009,10.538,100.0
3,2020-03-04,US,1,Alabama,NaN,01,149352,10.144,96.0
4,2020-03-05,US,1,Alabama,NaN,01,144109,10.982,104.0


In [33]:
float(dfm.loc[(dfm['date'] == '2020-04-03') & (dfm['fips'] == '36061')]['m50_index'])

0.0

In [34]:
# add m50_index to training dataframe
mobility = []
for i in tqdm(range(len(dfX))):
    date = dfX.at[i, 'date']
    county = dfX.at[i, 'fips']
    
    # we want the mobility 2 weeks prior because this is most correlated with deaths on a given date
    w2_prior = str_from_date(date_from_str(date) - dt.timedelta(weeks=2))
    
    rowm = dfm.loc[(dfm['date'] == w2_prior) & (dfm['fips'] == county)]
    if not rowm.empty:
        mobility.append(float(rowm['m50_index']))
    else:
        mobility.append(float('NaN'))
dfX['mobility'] = np.array(mobility)

100%|██████████| 73773/73773 [47:14<00:00, 26.02it/s]    


In [35]:
dfX

,index,fips,date,county,state,cases,deaths,DDFD,DW,DM,...,Total households!!Households with one or more people 65 years and over,Total households!!Average household size,RELATIONSHIP!!Population in households,Cardiovascular Disease (AAMC),Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC),Active Physicians per 100000 Population 2018 (AAMC),pclon10,pclat10,Days from Reopening,mobility
0,1445,6023,2020-05-21,Humboldt,California,0.0,2.0,0,3,21,...,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN,NaN
1,1446,6023,2020-05-22,Humboldt,California,3.0,0.0,1,4,22,...,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN,NaN
2,1447,6023,2020-05-23,Humboldt,California,2.0,0.0,2,5,23,...,17776.0,2.37,132266.0,7.887885,0.337,279.6,-124.087189,40.752982,NaN,NaN
3,27480,45063,2020-03-16,Lexington,South Carolina,1.0,1.0,0,0,16,...,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732,-49.0,103.0
4,27481,45063,2020-03-17,Lexington,South Carolina,1.0,0.0,1,1,17,...,34226.0,2.53,292142.0,14.797656,0.312,229.5,-81.213684,33.958732,-48.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73768,13947,13185,2020-05-19,Lowndes,Georgia,1.0,0.0,53,1,19,...,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,19.0,72.0
73769,13948,13185,2020-05-20,Lowndes,Georgia,17.0,0.0,54,2,20,...,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,20.0,83.0
73770,13949,13185,2020-05-21,Lowndes,Georgia,3.0,0.0,55,3,21,...,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,21.0,76.0
73771,13950,13185,2020-05-22,Lowndes,Georgia,3.0,0.0,56,4,22,...,10618.0,2.68,111914.0,6.767301,0.308,228.7,-83.280221,30.857006,22.0,87.0


In [36]:
# write final training dataframe to disk
path = "completed_master_X.csv"
dfX.to_csv(index=False, path_or_buf=path)